In [ ]:
import gensim
gensim.__version__

In [ ]:
import nltk
nltk.__version__

In [ ]:
from gensim.scripts.glove2word2vec import glove2word2vec
global_glove_file = '../CI_Policy_Annotation/Flask/static/glove.6B/glove.6B.50d.txt'
glove2word2vec(glove_input_file=global_glove_file, word2vec_output_file="gensim_glove_vectors.txt")

from gensim.models.keyedvectors import KeyedVectors
glove_model = KeyedVectors.load_word2vec_format("gensim_glove_vectors.txt", binary=False)

## Seed selection

In [5]:
import pandas as pd
df = pd.read_csv('fews-district-all.csv')
countries = df['country'].unique()

In [6]:
countries

array(['Ethiopia', 'South Sudan', 'Central African Republic', 'Sudan',
       'Democratic Republic of the Congo', 'Uganda', 'Kenya', 'Rwanda',
       'Tanzania', 'Burundi', 'Mozambique', 'Malawi', 'Zambia', 'Somalia',
       'Djibouti', 'Iilemi triangle', 'Chad', 'Abyei', 'Yemen', 'Haiti',
       'Guatemala', 'Honduras', 'El Salvador', 'Afghanistan',
       'Tajikistan', 'Angola', 'Zimbabwe', 'Madagascar', 'Congo',
       'Guinea', 'Mali', 'Senegal', 'Mauritania', 'Burkina Faso',
       'Nigeria', 'Niger', 'Cameroon', 'Sierra Leone', 'Liberia'],
      dtype=object)

In [ ]:
from collections import Counter

def bigram_sequence(text_lst):
    result = [a for ls in text_lst for a in zip(ls.split(" ")[:-1], ls.split(" ")[1:]) if 'food' in a or 'hunger' in a]
    return result

def trigram_sequence(text_lst):
    result = [a for ls in text_lst for a in zip(ls.split(" ")[:-2], ls.split(" ")[1:-1], ls.split(" ")[2:]) if 'food' in a or 'hunger' in a]
    return result

def get_words(country):
    words = Counter([])
    for t in range(25):
        # Post-processed Factiva avro files into csv
        df = pd.read_csv("./templates-v3/{}/tmp{}.csv".format(country, str(t).zfill(2)))
        for r in df.iterrows():
            unigrams = set(r[1]['body'].split())
            bigrams = set(bigram_sequence(r[1]['body'].split('.')))
            trigrams = set(trigram_sequence(r[1]['body'].split('.')))
            words.update(unigrams)
            words.update(bigrams)
            words.update(trigrams)
    return words

all_words = Counter([])
for country in countries:
    all_words.update(get_words(country))
select_df = pd.DataFrame()
select_df['seed'] = list(all_words)
select_df.to_csv('selected_words.csv')

In [ ]:
# unigrams, bigrams and trigrams with word 'food' and 'hunger'
seeds = pd.read_csv('selected_words.csv')
keys = ['hunger crises', 'famine', 'food insecurity']
seeds['distance'] = seeds['seed'].apply(lambda x: min([glove_model.wmdistance(x, k) for k in keys]))
# including the 3 keys, retain just the top 100
seeds.sort_values(by=['distance'])[:103].to_csv('seeds_with_distance.csv')

In [ ]:
seeds['distance'].describe()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from nltk.corpus import stopwords
import numpy as np
from nltk import word_tokenize, pos_tag, ne_chunk
from nltk import Tree
from nltk.stem import PorterStemmer 

#1211 causes from news and literature
for country in countries:
    tf_cause = {}
    num_causes = 0
    file_type = 'cause'
    # these csv files are results from running the semantic frame parsing algorithm on news and literature
    df = pd.read_csv('results/{}_seeded_{}_cluster.csv'.format(country, file_type), delimiter=',')
    j = 1
    cause_thr = 0
    for cause, cause_sim in zip(df[df['{}_cluster'.format(file_type)] == i][file_type],
                                df[df['{}_cluster'.format(file_type)] == i]['cause_similarity']):
        num_causes += 1
        tokenized_doc  = word_tokenize(cause)
        tagged_sentences = pos_tag(tokenized_doc)
        stem_cause = " ".join([ps.stem(w) for w in cause.split(" ")])
        c_pre = ''
        for tag in tagged_sentences:
            c = ps.stem(tag[0])
            t = tag[1]
            if c in stemmed_stopwords or (not c.isalpha()) or c in black_list:
                continue
            if t.startswith('JJ'):
                c_pre = c
                continue
            if c_pre:
                c = c_pre + ' ' + c
                c_pre = ''
            if t.startswith('NN') and c_pre == '':
                c_pre = c
            tf_cause[c] = cause_sim[0]               
    cause_df = pd.DataFrame.from_dict(tf_cause, orient='index', columns=['count'])
    cause_df = cause_df.reset_index()
    causes_df = pd.concat([causes_df, cause_df], axis=0)

# expand 
for word, count in all_words.items():
    if count <= 1000:
        continue
    for cause in causes_df['cause'].unique():
        if glove_model.wmdistance(cause.split(), word.split()) < 6:
            tf_cause = {'cause': word, 'dist': glove_model.wmdistance(cause.split(), word.split())}
            cause_df = pd.DataFrame.from_dict(tf_cause)
            cause_df = cause_df.reset_index()
            causes_df = pd.concat([causes_df, cause_df])

causes_df.to_csv('expanded_cluster_causes.csv')

In [ ]:
# Get p-value of granger test for each of the variables with FEWS score
sign_ind = [i for i,v in enumerate(list(causes_df['p_value']<0.01)) if v]
causes = [causes_df.loc[i, 'cause'] for i in sign_ind]
X_filtered = [X[i] for i in sign_ind]